# Main Method

- Step 1: Run the installations
- Step 2: Run the code
    - NgRok should print a public url after the execution
    - Update the public url in the API


### Installations

In [ ]:
!pip install torch>=1.4 diffusers==0.10.2 sentencepiece jedi FLAX transformers scipy ftfy accelerate pyngrok==4.1.1 flask_ngrok datasets

!ngrok authtoken <your-auth-token>

### Executable Code

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, render_template_string, request, jsonify


import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset

import base64
from io import BytesIO


# ======== TEXT TRANSFORMER ========
# Load Model
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


# ==== IMAGE TRANSFORMER ======

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")


# ===== Web Server with NgRok ===
app = Flask(__name__)
run_with_ngrok(app)


@app.route('/')
def initial():
  html = '''
     <form action="submit-prompt" method="post" id="submission-form">
      <input type="text" name="user_input" />
      <input type="submit" value="Submit" />
    </form>
  '''
  return render_template_string(html)


@app.route('/submit-prompt', methods=['POST'])
def generate():

    data = request.get_json()
    prompt = data.get('prompt', '')
    

    # generate image
    print(f"Generating an image of {prompt} ...")
    image = pipe(prompt).images[0]
    print("Image generated! Converting image ...")
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue())
    img_str = "data:image/png;base64," + str(img_str)[2:-1]

    # generate text
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    generated_output = model.generate(
        input_ids, do_sample=True, temperature=1.0, max_length=2500, num_return_sequences=1)
    generated_text = tokenizer.decode(
        generated_output[0], skip_special_tokens=True)

    print("Sending image and text ...")
    data = {
        "image_str": img_str, 
        "response": generated_text,
        "prompt": prompt
    }
    return jsonify(data)


if __name__ == '__main__':
    app.run()
